In [1]:
import pandas as pd
import requests
import json
import time
# import numpy as np
# import statsmodels.api as sm
# import statsmodels.formula.api as smf

# from sklearn.metrics import roc_auc_score, roc_curve #Revérifier si tous les import de cette ligne sont utilisés
# from sklearn.model_selection import train_test_split

# from sklearn.linear_model import LogisticRegression
# from sklearn import svm
# from sklearn.ensemble import AdaBoostClassifier
# from sklearn.ensemble import RandomForestClassifier

In [2]:
df = pd.read_csv("data/aaaa.csv", delimiter = ';')

In [3]:
df['URL_query'] = '' #Create column of URLs we can query with modules and APIs later
N = len(df)
for i in range(N):
    url = df.loc[i]['URLs']
    if 'https://' in url:
        df['URL_query'][i] = url
    else:
        new_url = "https://" + url
        df['URL_query'][i] = new_url

In [4]:
df["request"] = 0
for i in range(len(df["URL_query"])): #We check the accesses to each link and Fast check of URL access
    url = df["URL_query"][i]
    try:
        r = requests.get(url, auth=('user', 'pass'))
#         print(r.status_code)
        df["request"][i] = r.status_code
    except:
        df["request"][i] = "unreachable"
#         print("unreachable")

df.drop(df[df["request"] != 200].index, inplace = True)

df.reset_index(drop = True, inplace = True)

C:\Users\tahad\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\tahad\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\tahad\Anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


### /!\ Caution: Runtime of the following block exceeds 15 minutes

In [10]:
l = []
key = "wuro4lfosm0mszlksu5523ij4ca5rqxhiz54uz0c9mgnyb2ldvvemm9jj4jsk6zg85oy8z"
for i in range(len(df)):
    url = "https://whatcms.org/API/Tech?key=" + key + "&url=" + df['URL_query'][i]
    
    payload={}
    headers = {}
    
    response = requests.request("GET", url, headers=headers, data=payload)
    output = json.loads(response.text)
    l.append(output)
    time.sleep(11)

# Data integration

In [24]:
whatcms_df = pd.DataFrame(l)

# with open("l_tech.json","r") as f:
#     whatcms_df = pd.DataFrame(json.load(f))


whatcms_df.pop('request')
whatcms_df.pop('request_web')
whatcms_df.pop('last_checked')
whatcms_df.pop('result')

whatcms_df['URL'] = df['URL_query']

In [25]:
temp_res = [] #Out of the whatcms output I only pick the names of the CMS/tech used
N = len(whatcms_df)
for i in range(N):
    row = whatcms_df.results[i]
    sub_l = []
    for j in range(len(row)):
        sub_l.append(row[j]['name'])
    temp_res.append(sub_l)

In [26]:
list_tags = [] #Exhaustive list of all the CMS/Tech used to build the columns of our dataframe
for _ in temp_res:
    for x in _:
        if x not in list_tags:
            list_tags.append(x)
n_tags = len(list_tags)

index_cms = [] #Same as temp_res, but with the indexes of a CMS instead of its name
for i in range(N):
    row = []
    row_cms = temp_res[i]
    for j in row_cms:
        row.append(list_tags.index(j))
    index_cms.append(row)

In [27]:
sparse_tags = [] #We generate a sparse Table for later model purpose
for i in range(N):
    row = index_cms[i]
    l_row = [0]*n_tags
    for x in row:
        l_row[x] = 1
    sparse_tags.append(l_row)
Tags_df = pd.DataFrame(sparse_tags, columns=list_tags)

In [38]:
full_df = pd.concat([df,Tags_df],axis = 1)
full_df.drop(['URLs','request'], inplace = True, axis = 1)

In [41]:
full_df.to_csv('data/whatcms_df.csv')

In [42]:
full_df

,dropshipping?,URL_query,Shopify,Cloudflare,cdnjs,jQuery CDN,Google Hosted Libraries,Unpkg,WordPress,WooCommerce,...,Ruby,Java,PageFly,Microsoft Ajax Content Delivery Network,Ubuntu,BigCommerce,Nginx,OpenResty,Next.js,Node.js
0,oui,https://touttrive.fr,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,oui,https://en.g-heat.com,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,oui,https://ecolessive.fr,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,oui,https://jolipuzzle.com,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,oui,https://dalinner.com,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,Oui,https://www.clubfitwear.com/,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
73,Oui,https://notebooktherapy.com/,1,1,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
74,Oui,https://warmlydecor.com/,1,1,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
75,Oui,https://www.inspireuplift.com/,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
